In [1]:
# ============================================================================
# TASK 4: FINAL FORECASTING EXECUTION
# Ethiopia Financial Inclusion Forecasts 2025-2027
# ============================================================================

import sys
sys.path.append('./src')

In [2]:
from forecasting.core_models import FinancialInclusionForecaster
from forecasting.scenario_engine import ScenarioGenerator
from forecasting.uncertainty import UncertaintyQuantifier
from forecasting.visualization import ForecastVisualizer
from utils.data_loader import load_task_data

import pandas as pd
import matplotlib.pyplot as plt

In [3]:
print("🎯 TASK 4: FINANCIAL INCLUSION FORECASTING")
print("="*60)

🎯 TASK 4: FINANCIAL INCLUSION FORECASTING


In [ ]:
print("📂 LOADING DATA MANUALLY")
print("="*50)

try:
    # Try to load the enriched dataset directly
    df = pd.read_csv('data/processed/ethiopia_fi_enriched.csv', low_memory=False)
    print(f"✅ Loaded enriched dataset: {df.shape}")
    
    # Extract historical data for key indicators
    historical_data = []
    
    # Account Ownership
    acc_data = df[(df['indicator_code'] == 'ACC_OWNERSHIP') & 
                  (df['record_type'] == 'observation')]
    if not acc_data.empty:
        for _, row in acc_data.iterrows():
            # Extract year from date
            if pd.notna(row.get('observation_date')):
                try:
                    year = pd.to_datetime(row['observation_date']).year
                    value = row['value_numeric']
                    if pd.notna(year) and pd.notna(value):
                        historical_data.append({
                            'year': int(year),
                            'ACC_OWNERSHIP': float(value)
                        })
                except:
                    pass
    
    # Digital Payments
    dp_data = df[(df['indicator_code'] == 'USG_DIGITAL_PAYMENT') & 
                 (df['record_type'] == 'observation')]
    if not dp_data.empty:
        for _, row in dp_data.iterrows():
            if pd.notna(row.get('observation_date')):
                try:
                    year = pd.to_datetime(row['observation_date']).year
                    value = row['value_numeric']
                    if pd.notna(year) and pd.notna(value):
                        # Update existing record or create new
                        existing = next((item for item in historical_data 
                                       if item['year'] == year), None)
                        if existing:
                            existing['USG_DIGITAL_PAYMENT'] = float(value)
                        else:
                            historical_data.append({
                                'year': int(year),
                                'USG_DIGITAL_PAYMENT': float(value)
                            })
                except:
                    pass
    
    # Convert to DataFrame
    historical_df = pd.DataFrame(historical_data)
    
    # Ensure we have all Findex years
    findex_years = [2011, 2014, 2017, 2021, 2024]
    result_df = pd.DataFrame({'year': findex_years})
    
    # Merge with extracted data
    for indicator in ['ACC_OWNERSHIP', 'USG_DIGITAL_PAYMENT']:
        if indicator in historical_df.columns:
            result_df = result_df.merge(
                historical_df[['year', indicator]], 
                on='year', 
                how='left'
            )
        else:
            # Add default values if missing
            if indicator == 'ACC_OWNERSHIP':
                result_df[indicator] = [14.0, 22.0, 35.0, 46.0, 49.0]
            else:  # USG_DIGITAL_PAYMENT
                result_df[indicator] = [10.0, 18.0, 25.0, 35.0, 35.0]
    
    historical_data = result_df.sort_values('year').reset_index(drop=True)
    print(f"✅ Historical data shape: {historical_data.shape}")
    print(historical_data)
    
    # Load event matrix
    try:
        event_matrix = pd.read_csv('models/task3/event_indicator_association_matrix.csv')
        print(f"✅ Event matrix loaded: {event_matrix.shape}")
    except:
        print("⚠️ Creating sample event matrix")
        event_matrix = pd.DataFrame({
            'event_name': ['Telebirr Launch', 'M-Pesa Entry', 'QR System'],
            'ACC_OWNERSHIP_impact': [2.0, 1.5, 0.8],
            'USG_DIGITAL_PAYMENT_impact': [3.0, 2.5, 1.5]
        })
    
    # Create target data
    target_data = pd.DataFrame({
        'year': [2025, 2030],
        'ACC_OWNERSHIP': [70.0, 75.0],
        'USG_DIGITAL_PAYMENT': [45.0, 60.0]
    })
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("\n🔄 Using sample data instead...")
    
    # Fallback to sample data
    historical_data = pd.DataFrame({
        'year': [2011, 2014, 2017, 2021, 2024],
        'ACC_OWNERSHIP': [14.0, 22.0, 35.0, 46.0, 49.0],
        'USG_DIGITAL_PAYMENT': [10.0, 18.0, 25.0, 35.0, 35.0]
    })
    
    event_matrix = pd.DataFrame({
        'event_name': ['Telebirr Launch', 'M-Pesa Entry', 'QR System'],
        'ACC_OWNERSHIP_impact': [2.0, 1.5, 0.8],
        'USG_DIGITAL_PAYMENT_impact': [3.0, 2.5, 1.5]
    })
    
    target_data = pd.DataFrame({
        'year': [2025, 2030],
        'ACC_OWNERSHIP': [70.0, 75.0],
        'USG_DIGITAL_PAYMENT': [45.0, 60.0]
    })

print(f"\n✅ Data loaded successfully")
print(f"   Historical years: {list(historical_data['year'])}")
print(f"   Events: {len(event_matrix)}")

📂 LOADING DATA MANUALLY
❌ Error loading data: name 'pd' is not defined

🔄 Using sample data instead...


NameError: name 'pd' is not defined

In [4]:
# 2. Initialize models
forecaster = FinancialInclusionForecaster()
scenarios = ScenarioGenerator()
visualizer = ForecastVisualizer()

In [5]:
# 3. Generate forecasts
forecast_results = forecaster.generate_complete_forecasts(
    historical_data, event_matrix, target_data
)

NameError: name 'historical_data' is not defined

In [ ]:
# 4. Create scenarios
scenario_analysis = scenarios.generate_all_scenarios(forecast_results)

In [ ]:
# 5. Quantify uncertainty
uncertainty = UncertaintyQuantifier.calculate_all_uncertainty(scenario_analysis)

In [ ]:
# 6. Generate visualizations
visualizer.create_comprehensive_dashboard(forecast_results, scenario_analysis, uncertainty)
print("✅ All visualizations generated")

In [ ]:
# 7. Save results
forecaster.save_results('models/task4/')
print("📁 Results saved to models/task4/")

In [ ]:
# 8. Generate report
report = forecaster.generate_final_report(scenario_analysis, uncertainty)
print("📄 Final report generated")

In [ ]:
# Optional: Display summary
print("\n" + "="*60)
print("📊 FORECAST SUMMARY")
print("="*60)

if 'ACC_OWNERSHIP' in forecast_results:
    acc_forecasts = forecast_results['ACC_OWNERSHIP']['ensemble']['forecasts']
    print("Account Ownership Forecasts:")
    for year, value in acc_forecasts.items():
        print(f"  {year}: {value:.1f}%")
    
if 'ACC_OWNERSHIP' in forecast_results and 'target_gap' in forecast_results['ACC_OWNERSHIP']:
    gap_2025 = forecast_results['ACC_OWNERSHIP']['target_gap'].get(2025, {})
    if gap_2025:
        print(f"\nNFIS-II 2025 Target Gap: {gap_2025.get('gap_pp', 0):.1f}pp")

In [ ]:
# Optional: Display scenario ranges
print("\n📈 Scenario Ranges (2027):")
if scenario_analysis and 'ACC_OWNERSHIP' in scenario_analysis:
    acc_scenarios = scenario_analysis['ACC_OWNERSHIP']
    for scenario in ['pessimistic', 'baseline', 'optimistic']:
        if scenario in acc_scenarios:
            value = acc_scenarios[scenario]['forecasts'].get(2027, 0)
            print(f"  {scenario.title()}: {value:.1f}%")